# Running a single model

Here is a notebook for running a DYNAMITE model.

## Section 1

In [3]:
import sys
sys.path.insert(0,'../../')
import dynamite as dyn

In [5]:
fname = 'NGC6278_config.yaml'
c = dyn.config_reader.ConfigurationReaderYaml(fname, silent=True)

For now - replace the legacy directory with the version I have elsewhere, which is already compiled:

In [17]:
tmp = '/Users/pjethwa/Astro/Science/dynamite/triaxschwarz/triaxschwarzschild'
c.settings.legacy_settings['directory'] = tmp

In [18]:
parspace = dyn.parameter_space.ParameterSpace(c.system)
all_models = dyn.model.AllModels(parspace=parspace, settings=c.settings)
kw_executor = {'system':c.system,
               'legacy_directory':c.settings.legacy_settings['directory'],
               'executor_settings':c.settings.executor_settings}
executor_type = c.settings.executor_settings['type']
executor = getattr(dyn.executor, executor_type)(**kw_executor)

NGC6278_output/all_models.ecsv
making empty table attribute
I love Slurm!
I love Slurm even more!!


In [19]:
print('Parameter / value :')
for par in parspace:
    print(f'   {par.name} = {par.value}')

Parameter / value :
   mass = 5.0
   a = 0.001
   dc = 3.0
   f = 1.0
   q = 0.54
   p = 0.99
   u = 0.9999
   ml = 5.0


In [20]:
print('Parameter / value :')
for par in parspace:
    raw_value = par.value
    par_value = par.get_par_value_from_raw_value(raw_value)
    print(f'   {par.name} = {par_value}')

Parameter / value :
   mass = 100000.0
   a = 0.001
   dc = 1000.0
   f = 10.0
   q = 0.54
   p = 0.99
   u = 0.9999
   ml = 5.0


In [21]:
from astropy.table import Table

In [22]:
t = Table()
for par in parspace:
    raw_value = par.value
    par_value = par.get_par_value_from_raw_value(raw_value)
    t[par.name] = [par_value]
# extract 0th - i.e. the only - row from the table
parset = t[0]

print(parset)

  mass     a     dc    f    q    p     u     ml
-------- ----- ------ ---- ---- ---- ------ ---
100000.0 0.001 1000.0 10.0 0.54 0.99 0.9999 5.0


In [23]:
model = dyn.model.LegacySchwarzschildModel(
    system=c.system,
    settings=c.settings,
    parspace=parspace,
    executor=executor,
    parset=parset)

In [24]:
model.setup_directories()

In [25]:
model.get_orblib()

Calculating the orbit library for the proposed potential.
Orbit integration is finished.


In [26]:
model.get_weights()

Fit the orbit library to the kinematic data.
NNLS is finished.


In [27]:
model.chi2

3698.687185